In [2]:
import torch
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, AdamW
from torchvision.datasets import CocoCaptions
from torchvision.transforms import transforms
import timm

# 1. Set up models and tokenizer
vit_model = timm.create_model("vit_base_patch16_224_in21k", pretrained=True).eval()
gpt2_config = GPT2Config.from_pretrained("gpt2-medium")
gpt2_model = GPT2LMHeadModel(gpt2_config)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")

# Assuming the COCO dataset is available
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
coco_dataset = CocoCaptions(root="./datasets/", annFile="./datasets/annotations/", transform=transform)
dataloader = DataLoader(coco_dataset, batch_size=8, shuffle=True)

optimizer = AdamW(gpt2_model.parameters(), lr=1e-4)

# 2. Training Loop
for epoch in range(num_epochs):
    for images, captions in dataloader:
        optimizer.zero_grad()

        # Extract image features using ViT
        image_features = vit_model(images)

        # Tokenize captions and get input and target tokens
        input_ids = tokenizer(captions, return_tensors="pt", padding="max_length", max_length=50, truncation=True).input_ids
        target_ids = input_ids[:, 1:]
        input_ids = input_ids[:, :-1]

        # Combine image features with token input ids
        # Note: This is a simplification; you'd likely need to integrate image features more elegantly with the token inputs.
        combined_inputs = torch.cat([image_features, input_ids], dim=1)

        # Forward pass through GPT-2 model
        outputs = gpt2_model(input_ids=combined_inputs, labels=target_ids)
        loss = outputs.loss

        # Backward pass and optimizer step
        loss.backward()
        optimizer.step()

loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_COCO_annotations'